# Auto Arima
Dieses Notebook enthält zusätzliche Testläufe mit auto_arima von pmdarima zum Testen der Parameter für die SARIMAX Modelle

In [1]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pmdarima as pm
from google.colab import drive
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler

In [3]:
base_path = '/content/drive/My Drive/data_collection/'
weather_time_moer_file_name = 'weather_time_moer_2021_2024_DE_NO.csv'
weather_time_moer_filtered_file_name = 'weather_time_moer_filtered_2021_2024_DE_NO.csv'
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
start_date = pd.to_datetime('2023-09-01 00:00')
training_cutoff_date = pd.to_datetime('2024-02-22 23:00')

In [4]:
start_date = pd.to_datetime('2023-02-01 00:00')
training_cutoff_date = pd.to_datetime('2024-01-31 23:00')

In [ ]:
weather_time_moer = pd.read_csv(base_path + weather_time_moer_file_name, index_col='date')
de = weather_time_moer[weather_time_moer['country'] == "DE"]
no = weather_time_moer[weather_time_moer['country'] == "NO"]
de.index = pd.to_datetime(de.index)
no.index = pd.to_datetime(no.index)

In [ ]:
train_de = de[(de.index >= start_date) & (de.index <= training_cutoff_date)]
train_no = no[(no.index >= start_date) & (no.index <= training_cutoff_date)]
exog_variables = [
    'temperature', 'ghi', 'wind_speed', 'precipitation', 'hour_of_day', 'day_of_week',
    'day_of_year', 'is_holiday_or_weekend', 'season'
]
exog_train_de = train_de[exog_variables]
exog_train_no = train_no[exog_variables]

In [ ]:
exog_variables_short = ['hour_of_day', 'day_of_year']
exog_variables_short_2 = ['hour_of_day', 'day_of_week', 'day_of_year', 'is_holiday_or_weekend', 'season']
exog_train_de_short = train_de[exog_variables_short]
exog_train_no_short = train_no[exog_variables_short]
exog_train_de_short_2 = train_de[exog_variables_short_2]

In [ ]:
# automatically select best ARIMA model
model = pm.auto_arima(train_de['moer'],
                      exogenous=exog_train_de,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=3, max_q=3,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
model.summary()

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[24] intercept   : AIC=44191.699, Time=8.51 sec
 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=44311.031, Time=0.07 sec
 ARIMA(1,1,0)(1,0,0)[24] intercept   : AIC=44194.164, Time=4.94 sec
 ARIMA(0,1,1)(0,0,1)[24] intercept   : AIC=44201.084, Time=4.95 sec
 ARIMA(0,1,0)(0,0,0)[24]             : AIC=44309.032, Time=0.05 sec
 ARIMA(0,1,0)(0,0,1)[24] intercept   : AIC=44199.322, Time=5.00 sec
 ARIMA(0,1,0)(1,0,0)[24] intercept   : AIC=44192.457, Time=5.65 sec
 ARIMA(0,1,0)(2,0,1)[24] intercept   : AIC=inf, Time=39.61 sec
 ARIMA(0,1,0)(1,0,2)[24] intercept   : AIC=inf, Time=44.15 sec
 ARIMA(0,1,0)(0,0,2)[24] intercept   : AIC=44194.605, Time=17.73 sec
 ARIMA(0,1,0)(2,0,0)[24] intercept   : AIC=44192.579, Time=20.70 sec
 ARIMA(0,1,0)(2,0,2)[24] intercept   : AIC=inf, Time=67.09 sec
 ARIMA(1,1,0)(1,0,1)[24] intercept   : AIC=44193.382, Time=6.84 sec
 ARIMA(0,1,1)(1,0,1)[24] intercept   : AIC=44193.332, Time=8.52 sec
 ARIMA(1,1,1)(1,0,

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                 4248
Model:             SARIMAX(0, 1, 0)x(1, 0, [1], 24)   Log Likelihood              -22091.849
Date:                              Wed, 27 Mar 2024   AIC                          44189.699
Time:                                      12:26:19   BIC                          44208.761
Sample:                                  09-01-2023   HQIC                         44196.435
                                       - 02-24-2024                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.S.L24       0.3573      0.068      5.268      0.000       0.224       0.490
ma.S.L24      -0.1974      0.069     -2.850      0.004      -0.333      -0.062
sigma2      1930.2763      9.892    195.141      0.000    1910.889    1949.664
===================================================================================
Ljung-Box (L1) (Q):                   0.31   Jarque-Bera (JB):            230019.14
Prob(Q):                              0.58   Prob(JB):                         0.00
Heteroskedasticity (H):               0.84   Skew:                            -1.51
Prob(H) (two-sided):                  0.00   Kurtosis:                        38.93
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
model = pm.auto_arima(train_de['moer'],
                      exogenous=exog_train_de,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=47873.564, Time=12.09 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=48262.940, Time=0.06 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=43738.342, Time=19.47 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=45273.802, Time=5.39 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=68412.636, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=43881.661, Time=0.13 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=43750.965, Time=131.84 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=43744.637, Time=27.24 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=43748.168, Time=5.75 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=46353.910, Time=89.51 sec
 ARIMA(0,0,0)(1,0,0)[24] intercept   : AIC=47848.159, Time=8.84 sec
 ARIMA(2,0,0)(1,0,0)[24] intercept   : AIC=43705.048, Time=21.92 sec
 ARIMA(2,0,0)(0,0,0)[24] intercept   : AIC=43822.830, Time=0.28 sec
 ARIMA(2,0,0)(2,0,0)[24] intercept   : AIC=43718.569, Time=237.10 

In [ ]:
model = pm.auto_arima(train_de['moer'],
                      exogenous=exog_train_de_short,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=47873.568, Time=18.14 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=48262.940, Time=0.08 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=43749.361, Time=15.46 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=45273.802, Time=8.40 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=68412.636, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=43881.661, Time=0.19 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=43751.120, Time=157.09 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=43744.639, Time=34.02 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=43748.168, Time=8.34 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=46175.802, Time=136.56 sec
 ARIMA(1,0,0)(1,0,2)[24] intercept   : AIC=43742.579, Time=111.70 sec
 ARIMA(1,0,0)(0,0,2)[24] intercept   : AIC=43742.422, Time=34.07 sec
 ARIMA(0,0,0)(0,0,2)[24] intercept   : AIC=47854.716, Time=28.96 sec
 ARIMA(2,0,0)(0,0,2)[24] intercept   : AIC=43695.891, Time=39.

In [ ]:
# new training cutoff
model = pm.auto_arima(train_de['moer'],
                      exogenous=exog_train_de,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=47215.977, Time=20.19 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=47640.472, Time=0.08 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=43088.192, Time=27.34 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=44547.095, Time=12.09 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=67642.227, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=43254.471, Time=0.19 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=43103.210, Time=214.52 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=43095.528, Time=35.53 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=43099.278, Time=17.24 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=44709.025, Time=140.43 sec
 ARIMA(0,0,0)(1,0,0)[24] intercept   : AIC=47186.077, Time=15.55 sec
 ARIMA(2,0,0)(1,0,0)[24] intercept   : AIC=43025.429, Time=40.60 sec
 ARIMA(2,0,0)(0,0,0)[24] intercept   : AIC=43175.295, Time=0.41 sec
 ARIMA(2,0,0)(2,0,0)[24] intercept   : AIC=43041.972, Time=263

In [ ]:
# new training cutoff and exog short 2
model = pm.auto_arima(train_de['moer'],
                      exogenous=exog_train_de_short_2,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=47215.977, Time=22.89 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=47640.472, Time=0.10 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=43088.192, Time=30.21 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=44547.095, Time=13.98 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=67642.227, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=43254.471, Time=0.20 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=43103.210, Time=244.88 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=43095.528, Time=41.24 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=43099.278, Time=19.93 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=44709.025, Time=154.95 sec
 ARIMA(0,0,0)(1,0,0)[24] intercept   : AIC=47186.077, Time=16.90 sec
 ARIMA(2,0,0)(1,0,0)[24] intercept   : AIC=43025.429, Time=44.30 sec
 ARIMA(2,0,0)(0,0,0)[24] intercept   : AIC=43175.295, Time=0.44 sec
 ARIMA(2,0,0)(2,0,0)[24] intercept   : AIC=43041.972, Time=291

In [ ]:
# New training cutoff and new start date
model = pm.auto_arima(train_de['moer'],
                      exogenous=exog_train_de,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=9.23 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=13533.855, Time=0.05 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=12255.048, Time=5.77 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=12744.730, Time=3.34 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=20520.850, Time=0.11 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=12254.445, Time=0.09 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=12255.153, Time=1.61 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=18.74 sec
 ARIMA(2,0,0)(0,0,0)[24] intercept   : AIC=12246.268, Time=0.38 sec
 ARIMA(2,0,0)(1,0,0)[24] intercept   : AIC=12247.575, Time=8.02 sec
 ARIMA(2,0,0)(0,0,1)[24] intercept   : AIC=12247.611, Time=2.78 sec
 ARIMA(2,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=14.34 sec
 ARIMA(3,0,0)(0,0,0)[24] intercept   : AIC=12222.364, Time=0.23 sec
 ARIMA(3,0,0)(1,0,0)[24] intercept   : AIC=12223.795, Time=14.12 sec
 ARIMA(3,0,0)(0,0,1)

/usr/local/lib/python3.10/dist-packages/pmdarima/arima/_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(3,0,0)(1,0,1)[24] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
  File "/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/mlemodel.py", line 704, in fit
    mlefit = super(MLEModel, self).fit(start_params, method=method,
  File "/usr/local/lib/python3

 ARIMA(3,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=nan sec
 ARIMA(4,0,0)(0,0,0)[24] intercept   : AIC=12224.243, Time=0.18 sec
 ARIMA(3,0,1)(0,0,0)[24] intercept   : AIC=12224.354, Time=0.72 sec
 ARIMA(2,0,1)(0,0,0)[24] intercept   : AIC=12251.384, Time=0.89 sec
 ARIMA(4,0,1)(0,0,0)[24] intercept   : AIC=12226.229, Time=0.96 sec
 ARIMA(3,0,0)(0,0,0)[24]             : AIC=inf, Time=0.05 sec

Best model:  ARIMA(3,0,0)(0,0,0)[24] intercept
Total fit time: 105.088 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 1272
Model:               SARIMAX(3, 0, 0)   Log Likelihood               -6106.182
Date:                Mon, 08 Apr 2024   AIC                          12222.364
Time:                        10:14:29   BIC                          12248.106
Sample:                    01-01-2024   HQIC                         12232.033
                         - 02-22-2024              

In [ ]:
# New training cutoff and new start date
model = pm.auto_arima(train_de['moer'],
                      exogenous=exog_train_de,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=10.72 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=13533.855, Time=0.06 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=12255.048, Time=6.14 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=12744.730, Time=4.25 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=20520.850, Time=0.14 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=12254.445, Time=0.09 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=12255.153, Time=1.68 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=20.07 sec
 ARIMA(2,0,0)(0,0,0)[24] intercept   : AIC=12246.268, Time=0.37 sec
 ARIMA(2,0,0)(1,0,0)[24] intercept   : AIC=12247.575, Time=8.36 sec
 ARIMA(2,0,0)(0,0,1)[24] intercept   : AIC=12247.611, Time=2.72 sec
 ARIMA(2,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=14.26 sec
 ARIMA(3,0,0)(0,0,0)[24] intercept   : AIC=12222.364, Time=0.24 sec
 ARIMA(3,0,0)(1,0,0)[24] intercept   : AIC=12223.795, Time=14.26 sec
 ARIMA(3,0,0)(0,0,1

/usr/local/lib/python3.10/dist-packages/pmdarima/arima/_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(3,0,0)(1,0,1)[24] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
  File "/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/mlemodel.py", line 704, in fit
    mlefit = super(MLEModel, self).fit(start_params, method=method,
  File "/usr/local/lib/python3

 ARIMA(3,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=nan sec
 ARIMA(4,0,0)(0,0,0)[24] intercept   : AIC=12224.243, Time=0.16 sec
 ARIMA(3,0,1)(0,0,0)[24] intercept   : AIC=12224.354, Time=0.70 sec
 ARIMA(2,0,1)(0,0,0)[24] intercept   : AIC=12251.384, Time=0.87 sec
 ARIMA(4,0,1)(0,0,0)[24] intercept   : AIC=12226.229, Time=0.95 sec
 ARIMA(3,0,0)(0,0,0)[24]             : AIC=inf, Time=0.05 sec

Best model:  ARIMA(3,0,0)(0,0,0)[24] intercept
Total fit time: 110.043 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 1272
Model:               SARIMAX(3, 0, 0)   Log Likelihood               -6106.182
Date:                Mon, 08 Apr 2024   AIC                          12222.364
Time:                        09:42:42   BIC                          12248.106
Sample:                    01-01-2024   HQIC                         12232.033
                         - 02-22-2024              

In [ ]:
# New training cutoff and new start date
model = pm.auto_arima(train_de['moer'],
                      exogenous=exog_train_de,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=1,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[24] intercept   : AIC=inf, Time=6.38 sec
 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=12378.476, Time=0.06 sec
 ARIMA(1,1,0)(1,0,0)[24] intercept   : AIC=12381.472, Time=1.38 sec
 ARIMA(0,1,1)(0,0,1)[24] intercept   : AIC=12381.527, Time=2.45 sec
 ARIMA(0,1,0)(0,0,0)[24]             : AIC=12376.476, Time=0.04 sec
 ARIMA(0,1,0)(1,0,0)[24] intercept   : AIC=12380.240, Time=0.82 sec
 ARIMA(0,1,0)(0,0,1)[24] intercept   : AIC=12380.263, Time=1.15 sec
 ARIMA(1,1,0)(0,0,0)[24] intercept   : AIC=12379.833, Time=0.08 sec
 ARIMA(0,1,1)(0,0,0)[24] intercept   : AIC=12379.853, Time=0.08 sec
 ARIMA(1,1,1)(0,0,0)[24] intercept   : AIC=12381.105, Time=0.38 sec

Best model:  ARIMA(0,1,0)(0,0,0)[24]          
Total fit time: 12.853 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 1272
Model:               SARIMAX(0, 1, 0)   Log Li

In [ ]:
# New training cutoff and new start date
model = pm.auto_arima(train_de['moer'],
                      exogenous=exog_train_de,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      D=1,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=9.65 sec
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=13893.759, Time=0.23 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=12473.964, Time=11.97 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=8.95 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=13891.775, Time=0.16 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=12832.893, Time=0.73 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=12346.852, Time=33.87 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=32.21 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=13.29 sec
 ARIMA(0,0,0)(2,1,0)[24] intercept   : AIC=13524.257, Time=40.48 sec
 ARIMA(2,0,0)(2,1,0)[24] intercept   : AIC=12341.400, Time=43.16 sec
 ARIMA(2,0,0)(1,1,0)[24] intercept   : AIC=12467.309, Time=14.54 sec
 ARIMA(2,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=34.02 sec
 ARIMA(2,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=16.60 sec
 ARIMA(3,0,0)(2,1,0)[24] interce

In [ ]:
model = pm.auto_arima(train_no['moer'],
                      exogenous=exog_train_no,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=14.61 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=42722.576, Time=0.06 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=37499.579, Time=13.56 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=39219.484, Time=7.67 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=59320.136, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=38303.863, Time=0.15 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=37166.035, Time=153.35 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=37180.333, Time=129.36 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=37022.188, Time=22.58 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=37822.277, Time=10.05 sec
 ARIMA(1,0,0)(1,0,2)[24] intercept   : AIC=37098.183, Time=61.72 sec
 ARIMA(1,0,0)(0,0,2)[24] intercept   : AIC=37538.455, Time=30.66 sec
 ARIMA(1,0,0)(2,0,2)[24] intercept   : AIC=inf, Time=98.75 sec
 ARIMA(2,0,0)(1,0,1)[24] intercept   : AIC=37121.073, Time=25.86 sec
 ARIM

In [ ]:
model = pm.auto_arima(train_no['moer'],
                      exogenous=exog_train_no,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=10, max_q=10,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=14.85 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=42722.576, Time=0.06 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=37499.579, Time=13.97 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=39219.484, Time=7.93 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=59320.136, Time=0.08 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=38303.863, Time=0.15 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=37166.035, Time=153.47 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=37180.333, Time=129.24 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=37022.188, Time=22.51 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=37822.277, Time=10.03 sec
 ARIMA(1,0,0)(1,0,2)[24] intercept   : AIC=37098.183, Time=60.76 sec
 ARIMA(1,0,0)(0,0,2)[24] intercept   : AIC=37538.455, Time=30.90 sec
 ARIMA(1,0,0)(2,0,2)[24] intercept   : AIC=inf, Time=98.51 sec
 ARIMA(2,0,0)(1,0,1)[24] intercept   : AIC=37121.073, Time=25.83 sec
 ARIM

In [ ]:
model = pm.auto_arima(train_no['moer'],
                      exogenous=exog_train_no,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0, D=1,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=29.12 sec
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=40558.743, Time=0.60 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=37523.068, Time=23.88 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=37757.386, Time=21.53 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=40556.779, Time=0.40 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=38614.040, Time=2.33 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=37181.419, Time=109.79 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=124.54 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=38.43 sec
 ARIMA(0,0,0)(2,1,0)[24] intercept   : AIC=39638.335, Time=60.35 sec
 ARIMA(2,0,0)(2,1,0)[24] intercept   : AIC=37177.832, Time=78.21 sec
 ARIMA(2,0,0)(1,1,0)[24] intercept   : AIC=37521.236, Time=24.38 sec
 ARIMA(2,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=127.88 sec
 ARIMA(2,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=46.07 sec
 ARIMA(3,0,0)(2,1,0)[

In [ ]:
# new training cutoff
model = pm.auto_arima(train_no['moer'],
                      exogenous=exog_train_no,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=40206.233, Time=26.36 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=42274.503, Time=0.08 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=37104.276, Time=23.15 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=38805.723, Time=12.30 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=58652.691, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=37899.357, Time=0.22 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=36798.427, Time=227.97 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=45540.634, Time=64.70 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=37071.976, Time=40.41 sec
 ARIMA(0,0,0)(2,0,0)[24] intercept   : AIC=inf, Time=107.64 sec
 ARIMA(2,0,0)(2,0,0)[24] intercept   : AIC=36752.333, Time=283.51 sec
 ARIMA(2,0,0)(1,0,0)[24] intercept   : AIC=37099.971, Time=57.80 sec
 ARIMA(2,0,0)(2,0,1)[24] intercept   : AIC=39497.170, Time=192.57 sec
 ARIMA(2,0,0)(1,0,1)[24] intercept   : AIC=36918.850, Time=42.78 

In [ ]:
model = pm.auto_arima(train_no['moer'],
                      exogenous=exog_train_no,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[24] intercept   : AIC=37169.083, Time=10.86 sec
 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=38319.811, Time=0.10 sec
 ARIMA(1,1,0)(1,0,0)[24] intercept   : AIC=37605.236, Time=8.02 sec
 ARIMA(0,1,1)(0,0,1)[24] intercept   : AIC=37837.895, Time=8.36 sec
 ARIMA(0,1,0)(0,0,0)[24]             : AIC=38317.811, Time=0.07 sec
 ARIMA(0,1,0)(0,0,1)[24] intercept   : AIC=37933.354, Time=6.32 sec
 ARIMA(0,1,0)(1,0,0)[24] intercept   : AIC=37718.899, Time=5.37 sec
 ARIMA(0,1,0)(2,0,1)[24] intercept   : AIC=37167.751, Time=39.58 sec
 ARIMA(0,1,0)(2,0,0)[24] intercept   : AIC=37404.105, Time=22.50 sec
 ARIMA(0,1,0)(2,0,2)[24] intercept   : AIC=37166.878, Time=59.06 sec
 ARIMA(0,1,0)(1,0,2)[24] intercept   : AIC=37167.857, Time=44.42 sec
 ARIMA(1,1,0)(2,0,2)[24] intercept   : AIC=37030.655, Time=80.93 sec
 ARIMA(1,1,0)(1,0,2)[24] intercept   : AIC=37030.659, Time=62.30 sec
 ARIMA(1,1,0)(2,0,1)[24] intercept   : AIC=37030.637, Time=59.09 s

Filtered data auto arima

In [5]:
weather_time_moer_filtered = pd.read_csv(base_path + weather_time_moer_filtered_file_name, index_col='date')
de_filtered = weather_time_moer_filtered[weather_time_moer_filtered['country'] == "DE"]
no_filtered = weather_time_moer_filtered[weather_time_moer_filtered['country'] == "NO"]
de_filtered.index = pd.to_datetime(de_filtered.index)
no_filtered.index = pd.to_datetime(no_filtered.index)

In [6]:
train_de_filtered = de_filtered[(de_filtered.index >= start_date) & (de_filtered.index <= training_cutoff_date)]
train_no_filtered = no_filtered[(no_filtered.index >= start_date) & (no_filtered.index <= training_cutoff_date)]
exog_train_de_filtered = train_de_filtered[exog_variables]
exog_train_no_filtered = train_no_filtered[exog_variables]

In [ ]:
model = pm.auto_arima(train_de_filtered['moer'],
                      exogenous=exog_train_de_filtered,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=35115.643, Time=40.61 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=36472.905, Time=0.09 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=31924.206, Time=49.89 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=33372.021, Time=10.12 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=64130.747, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=32261.513, Time=0.20 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=inf, Time=783.72 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=inf, Time=42.52 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=32002.590, Time=23.57 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=inf, Time=56.29 sec
 ARIMA(0,0,0)(1,0,0)[24] intercept   : AIC=36428.609, Time=13.10 sec
 ARIMA(2,0,0)(1,0,0)[24] intercept   : AIC=inf, Time=31.78 sec
 ARIMA(1,0,1)(1,0,0)[24] intercept   : AIC=inf, Time=21.01 sec
 ARIMA(0,0,1)(1,0,0)[24] intercept   : AIC=inf, Time=33.50 sec


/usr/local/lib/python3.10/dist-packages/pmdarima/arima/_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(2,0,1)(1,0,0)[24] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
  File "/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/mlemodel.py", line 704, in fit
    mlefit = super(MLEModel, self).fit(start_params, method=method,
  File "/usr/local/lib/python3

 ARIMA(2,0,1)(1,0,0)[24] intercept   : AIC=inf, Time=nan sec
 ARIMA(1,0,0)(1,0,0)[24]             : AIC=inf, Time=8.97 sec

Best model:  ARIMA(1,0,0)(1,0,0)[24] intercept
Total fit time: 1151.959 seconds
                                     SARIMAX Results                                      
Dep. Variable:                                  y   No. Observations:                 3975
Model:             SARIMAX(1, 0, 0)x(1, 0, 0, 24)   Log Likelihood              -15958.103
Date:                            Mon, 08 Apr 2024   AIC                          31924.206
Time:                                    15:21:17   BIC                          31949.357
Sample:                                         0   HQIC                         31933.124
                                           - 3975                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025  

In [ ]:
model = pm.auto_arima(train_de_filtered['moer'],
                      exogenous=exog_train_de_filtered,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=10, max_q=10,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True,
                      error_action="ignore")
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[24] intercept   : AIC=inf, Time=25.42 sec
 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=69120.643, Time=0.17 sec
 ARIMA(1,1,0)(1,0,0)[24] intercept   : AIC=67390.275, Time=13.02 sec
 ARIMA(0,1,1)(0,0,1)[24] intercept   : AIC=68060.013, Time=12.74 sec
 ARIMA(0,1,0)(0,0,0)[24]             : AIC=69118.643, Time=0.11 sec
 ARIMA(1,1,0)(0,0,0)[24] intercept   : AIC=69114.681, Time=0.29 sec
 ARIMA(1,1,0)(2,0,0)[24] intercept   : AIC=66833.027, Time=45.80 sec
 ARIMA(1,1,0)(2,0,1)[24] intercept   : AIC=inf, Time=130.57 sec
 ARIMA(1,1,0)(1,0,1)[24] intercept   : AIC=inf, Time=32.79 sec
 ARIMA(0,1,0)(2,0,0)[24] intercept   : AIC=67181.443, Time=33.42 sec
 ARIMA(2,1,0)(2,0,0)[24] intercept   : AIC=66800.422, Time=63.10 sec
 ARIMA(2,1,0)(1,0,0)[24] intercept   : AIC=67389.113, Time=18.94 sec
 ARIMA(2,1,0)(2,0,1)[24] intercept   : AIC=inf, Time=173.60 sec
 ARIMA(2,1,0)(1,0,1)[24] intercept   : AIC=inf, Time=48.83 sec
 ARIMA(3,1,0)(2,0,0)[2

In [ ]:
model = pm.auto_arima(train_de_filtered['moer'],
                      exogenous=exog_train_de_filtered,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=10, max_q=10,
                      d=0, D=1,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True,
                      error_action="ignore")
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=58.73 sec
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=73038.660, Time=2.18 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=66800.599, Time=48.52 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=58.71 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=73036.736, Time=1.74 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=68807.791, Time=4.77 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=66167.059, Time=217.42 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=248.98 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=88.62 sec
 ARIMA(0,0,0)(2,1,0)[24] intercept   : AIC=70963.250, Time=84.98 sec
 ARIMA(2,0,0)(2,1,0)[24] intercept   : AIC=66114.352, Time=215.89 sec
 ARIMA(2,0,0)(1,1,0)[24] intercept   : AIC=66742.535, Time=43.84 sec
 ARIMA(2,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=345.66 sec
 ARIMA(2,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=124.33 sec


In [ ]:
model = pm.auto_arima(train_no_filtered['moer'],
                      exogenous=exog_train_no_filtered,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=10, max_q=10,
                      d=0, D=1,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True,
                      error_action="ignore")
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=26.01 sec
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=76124.191, Time=1.10 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=70476.251, Time=37.67 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=70383.805, Time=26.86 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=76122.205, Time=0.78 sec
 ARIMA(0,0,1)(0,1,0)[24] intercept   : AIC=73833.936, Time=3.89 sec
 ARIMA(0,0,1)(1,1,1)[24] intercept   : AIC=inf, Time=40.81 sec
 ARIMA(0,0,1)(0,1,2)[24] intercept   : AIC=inf, Time=77.59 sec
 ARIMA(0,0,1)(1,1,0)[24] intercept   : AIC=71560.676, Time=23.06 sec
 ARIMA(0,0,1)(1,1,2)[24] intercept   : AIC=inf, Time=180.75 sec
 ARIMA(0,0,0)(0,1,1)[24] intercept   : AIC=73130.229, Time=15.96 sec
 ARIMA(1,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=35.37 sec
 ARIMA(0,0,2)(0,1,1)[24] intercept   : AIC=69654.403, Time=36.61 sec
 ARIMA(0,0,2)(0,1,0)[24] intercept   : AIC=73199.056, Time=6.06 sec
 ARIMA(0,0,2)(1,1,1)[24]

In [8]:
model = pm.auto_arima(train_no_filtered['moer'],
                      exogenous=exog_train_no_filtered,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=10, max_q=10,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True,
                      error_action="ignore")
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=75303.887, Time=64.68 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=78711.108, Time=0.17 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=70245.849, Time=40.53 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=72886.265, Time=21.98 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=121611.824, Time=0.10 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=71712.128, Time=0.55 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=69438.987, Time=476.28 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=inf, Time=403.83 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=70190.146, Time=71.39 sec
 ARIMA(0,0,0)(2,0,0)[24] intercept   : AIC=79214.734, Time=217.54 sec
 ARIMA(2,0,0)(2,0,0)[24] intercept   : AIC=69596.082, Time=700.61 sec
 ARIMA(1,0,1)(2,0,0)[24] intercept   : AIC=69633.996, Time=779.39 sec
 ARIMA(0,0,1)(2,0,0)[24] intercept   : AIC=76143.113, Time=421.53 sec
 ARIMA(2,0,1)(2,0,0)[24] intercept   : AIC=inf, Time=688.08 se

In [ ]:
model = pm.auto_arima(train_no_filtered['moer'],
                      exogenous=exog_train_no_filtered,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=10, max_q=10,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True,
                      error_action="ignore")
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[24] intercept   : AIC=70985.682, Time=18.05 sec
 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=72909.445, Time=0.42 sec
 ARIMA(1,1,0)(1,0,0)[24] intercept   : AIC=71373.016, Time=12.67 sec
 ARIMA(0,1,1)(0,0,1)[24] intercept   : AIC=71625.258, Time=13.18 sec
 ARIMA(0,1,0)(0,0,0)[24]             : AIC=72907.445, Time=0.10 sec
 ARIMA(0,1,0)(0,0,1)[24] intercept   : AIC=72243.870, Time=10.32 sec
 ARIMA(0,1,0)(1,0,0)[24] intercept   : AIC=71855.350, Time=8.53 sec
 ARIMA(0,1,0)(2,0,1)[24] intercept   : AIC=70981.964, Time=62.05 sec
 ARIMA(0,1,0)(2,0,0)[24] intercept   : AIC=71176.409, Time=40.09 sec
 ARIMA(0,1,0)(2,0,2)[24] intercept   : AIC=70950.637, Time=77.76 sec
 ARIMA(0,1,0)(1,0,2)[24] intercept   : AIC=70983.885, Time=73.13 sec
 ARIMA(1,1,0)(2,0,2)[24] intercept   : AIC=70413.254, Time=120.15 sec
 ARIMA(1,1,0)(1,0,2)[24] intercept   : AIC=70441.198, Time=76.87 sec
 ARIMA(1,1,0)(2,0,1)[24] intercept   : AIC=70441.040, Time=73.

In [ ]:
model = pm.auto_arima(train_no_filtered['moer'],
                      exogenous=exog_train_no_filtered,
                      m=24,
                      seasonal=True,
                      start_p=0, start_q=0,
                      max_p=5, max_q=5,
                      d=0,
                      trace=True,
                      suppress_warnings=True,
                      stepwise=True)
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=33930.498, Time=30.20 sec
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=34617.526, Time=0.08 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=31688.202, Time=16.00 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=32517.199, Time=10.67 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=53448.337, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=31911.900, Time=0.19 sec


/usr/local/lib/python3.10/dist-packages/pmdarima/arima/_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(1,0,0)(2,0,0)[24] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/usr/local/lib/python3.10/dist-packages/pmdarima/arima/arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
  File "/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/mlemodel.py", line 704, in fit
    mlefit = super(MLEModel, self).fit(start_params, method=method,
  File "/usr/local/lib/python3

 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=inf, Time=nan sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=31668.621, Time=42.93 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=31748.352, Time=25.60 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=35863.180, Time=78.49 sec
 ARIMA(1,0,0)(1,0,2)[24] intercept   : AIC=31928.971, Time=153.50 sec
 ARIMA(1,0,0)(0,0,2)[24] intercept   : AIC=31640.152, Time=63.11 sec
 ARIMA(0,0,0)(0,0,2)[24] intercept   : AIC=33605.993, Time=20.91 sec
 ARIMA(2,0,0)(0,0,2)[24] intercept   : AIC=31635.989, Time=95.13 sec
 ARIMA(2,0,0)(0,0,1)[24] intercept   : AIC=31741.127, Time=29.59 sec
 ARIMA(2,0,0)(1,0,2)[24] intercept   : AIC=33761.140, Time=138.69 sec
 ARIMA(2,0,0)(1,0,1)[24] intercept   : AIC=34220.361, Time=46.08 sec
 ARIMA(3,0,0)(0,0,2)[24] intercept   : AIC=31637.686, Time=105.49 sec
 ARIMA(2,0,1)(0,0,2)[24] intercept   : AIC=31637.416, Time=46.60 sec
 ARIMA(1,0,1)(0,0,2)[24] intercept   : AIC=31640.397, Time=34.91 sec
 ARIMA(3,0,1)(0,0,2)[24] intercept   : 